In [16]:
import pandas as pd
import numpy as n
import os

In [17]:
def load_data_microbiomeHD(address_directory, output_root = False, id = 'Sam_id', covar_l = []):
    ### note that due to the complexity of metadata, the current microbiomeHD loading does 
    ### not take into account the covariates other than batches and diseaseStates
    ### so default vars_use will just be Dataset
    subdir_names = os.listdir(address_directory)
    subdir_names = [result for result in subdir_names if "results" in result]
    count_data_l = []
    intersect_taxa = []
    metadata_l = []
    crc_baxter_ids = None 
    
    for subdir in subdir_names:
        ## 1. get the otu table part
        # get the otu file
        subdir_path = address_directory + '/' + subdir
        current_RDP_names = os.listdir(subdir_path + '/RDP')
        current_dbotu_count_data_path = [result for result in current_RDP_names if "100.denovo.rdp_assigned" in result][0]
        current_dbotu_count_data = pd.read_csv(subdir_path+'/RDP/'+current_dbotu_count_data_path, delimiter='\t', index_col=0)
        current_taxa = list(current_dbotu_count_data.index)
        # set index with genus level
        current_taxa = [";".join(taxa.split(';')[:-2]) for taxa in current_taxa]
        print(current_dbotu_count_data.shape)
        current_dbotu_count_data.index = current_taxa
        print(current_dbotu_count_data.shape)
        # remove duplicate indexes by summing up rows with the same index
        current_dbotu_count_data = current_dbotu_count_data.groupby(level=0).sum()
        
        # save dataframe and feature list
        count_data_l.append(current_dbotu_count_data)
        
        if intersect_taxa == []:
            intersect_taxa = current_taxa
        else:
            intersect_taxa = list(set(intersect_taxa).intersection(current_taxa))
        
        # Output how many taxonomic units in each OTU table are contained in intersect_taxa
        num_intersecting_taxa = sum(current_dbotu_count_data.index.isin(intersect_taxa))
        print(f"{subdir} OTU table contains {num_intersecting_taxa} genus-level taxa that are in intersect_taxa.")


        ## 2. get the metadata
        # get the metadata file
        current_files_names = os.listdir(subdir_path)
        current_metadata_path = subdir_path + '/' + [result for result in current_files_names if "metadata" in result][0]
        print("metadata path", current_metadata_path)
        current_metadata = pd.read_csv(current_metadata_path, delimiter='\t', index_col=0, encoding='ISO-8859-1')['DiseaseState'].to_frame()
        current_metadata['Dataset'] = ["_".join(subdir.split("_")[:-1])]*current_metadata.shape[0]
        print(current_metadata.shape)

        # save crc_baxter_results Sample IDs
        if 'crc_baxter_results' in subdir:
            crc_baxter_ids = set(current_metadata.index)

        # get covariates if exists
        if covar_l != []:
            current_covars = pd.read_csv(current_metadata_path, delimiter='\t', index_col=0, encoding='ISO-8859-1')[covar_l]
            current_metadata = pd.concat([current_metadata, current_covars], axis=1)
        metadata_l.append(current_metadata)

    # intersect count data list
    intersect_count_data_l = [count_data[count_data.index.isin(intersect_taxa)] for count_data in count_data_l]

    # generate results
    combined_countdf = pd.concat(intersect_count_data_l, axis=1)
    combined_countdf = combined_countdf.dropna().T
    combined_metadata = pd.concat(metadata_l)
    combined_metadata[id] = list(combined_metadata.index)  # the default IDCol for microbiomeHD will be Sam_id

    data_mat, meta_data = preprocess(combined_countdf, combined_metadata, id, covar_l)

    # ensure that the sample ids are correctly aligned in metadata and count_table
    data_mat_ids = set(data_mat.index)
    meta_data_ids = set(meta_data.index)
    intersection_ids = data_mat_ids.intersection(meta_data_ids)

    # check if Sample IDs of crc_baxter_results in intersection_ids 
    if crc_baxter_ids is not None:
        crc_baxter_intersection = crc_baxter_ids.intersection(intersection_ids)
        if len(crc_baxter_intersection) == 0:
            print("Warning: No Sample IDs from crc_baxter_results were retained after merging.")
        else:
            print(f"Sample IDs from crc_baxter_results retained: {len(crc_baxter_intersection)} out of {len(crc_baxter_ids)}")

    # drop rows where indexes are not overlapping
    data_mat_non_intersecting = [id for id in data_mat_ids if id not in intersection_ids]
    data_mat = data_mat.drop(data_mat_non_intersecting)
    meta_data_non_intersecting = [id for id in meta_data_ids if id not in intersection_ids]
    meta_data = meta_data.drop(meta_data_non_intersecting)
    data_mat = data_mat.reindex(intersection_ids)
    meta_data = meta_data.reindex(intersection_ids)

    # save stuff if needed
    if output_root != False:
        data_mat.to_csv(output_root + "_count_data.csv")
        meta_data.to_csv(output_root + "_meta_data.csv", index=False)
    return data_mat, meta_data


In [18]:
def preprocess(data_mat, meta_data, IDCol, covar_l = []):
    initial_crc_baxter_count = meta_data[meta_data['Dataset'] == 'crc_baxter'].shape[0]
    print(f"Initial crc_baxter_results samples: {initial_crc_baxter_count}")

    # Step 1: Remove samples with all zeros
    data_mat = data_mat.loc[~(data_mat==0).all(axis=1)]
    kept_samples = data_mat.index
    meta_data = meta_data[meta_data[IDCol].isin(kept_samples)]
    
    crc_baxter_after_step1 = meta_data[meta_data['Dataset'] == 'crc_baxter'].shape[0]
    print(f"After Step 1 (remove all-zero samples): {crc_baxter_after_step1} crc_baxter_results samples retained")

    # Step 2: Remove features with all zeros
    col_names = list(data_mat.columns)
    col_sums = data_mat.sum(axis = 1)
    removable_feature_names = [col_names[index] for index, col_sum in enumerate(col_sums) if col_sum==0]
    data_mat.drop(removable_feature_names, axis=1, inplace=True)
    
    # Step 3: Remove samples with missing values for each covariate
    for covar in covar_l:
        meta_data = meta_data[meta_data[covar].notna()]
    
    crc_baxter_after_step3 = meta_data[meta_data['Dataset'] == 'crc_baxter'].shape[0]
    print(f"After Step 3 (remove samples with missing covariates): {crc_baxter_after_step3} crc_baxter_results samples retained")

    # Step 4: Ensure data_mat and meta_data alignment
    data_mat = data_mat.loc[meta_data[IDCol]]
    
    # Step 5: Final feature removal with all zeros
    col_names = list(data_mat.columns)
    col_sums = data_mat.sum(axis = 1)
    removable_feature_names = [col_names[index] for index, col_sum in enumerate(col_sums) if col_sum==0]
    data_mat.drop(removable_feature_names, axis=1, inplace=True)

    crc_baxter_final = meta_data[meta_data['Dataset'] == 'crc_baxter'].shape[0]
    print(f"Final crc_baxter_results samples retained: {crc_baxter_final}")

    return data_mat, meta_data

In [19]:
overall_path = '/Users/hehehe/Desktop/CRC/'
output_dir_path = overall_path+'/MOSAIC/microbiomeHD Merge/intermediate_microbiomeHD'
address_directory = overall_path+'/microbiomeHD'
data_mat, meta_data = load_data_microbiomeHD(address_directory, output_dir_path)

(1617, 43)
(1617, 43)
crc_xiang_results OTU table contains 136 genus-level taxa that are in intersect_taxa.
metadata path /Users/hehehe/Desktop/CRC//microbiomeHD/crc_xiang_results/crc_xiang.metadata.txt
(43, 2)
(837, 102)
(837, 102)
crc_zhao_results OTU table contains 80 genus-level taxa that are in intersect_taxa.
metadata path /Users/hehehe/Desktop/CRC//microbiomeHD/crc_zhao_results/crc_zhao.metadata.txt
(102, 2)
(82665, 129)
(82665, 129)
crc_zeller_results OTU table contains 78 genus-level taxa that are in intersect_taxa.
metadata path /Users/hehehe/Desktop/CRC//microbiomeHD/crc_zeller_results/crc_zeller.metadata.txt
(156, 2)
(122510, 490)
(122510, 490)
crc_baxter_results OTU table contains 76 genus-level taxa that are in intersect_taxa.
metadata path /Users/hehehe/Desktop/CRC//microbiomeHD/crc_baxter_results/crc_baxter.metadata.txt
(490, 2)
(70846, 92)
(70846, 92)
crc_zackular_results OTU table contains 76 genus-level taxa that are in intersect_taxa.
metadata path /Users/hehehe/Des

## crc_baxter_results are all removed in 'preprocess', 
## and in this Step0 we use 'load_data_microbiomeHD' which contains 'preprocess', 
## so there's no crc_baxter_results in 'microbiomeHD Merge'